In [13]:
import keyboard
import time 
import webbrowser


a = 1824

while a < 19000:

    a = a+1
    siteExtension = "/download/action/?type=schematic"
    url = ("www.minecraft-schematics.com/schematic/" + str(a) + siteExtension)
    webbrowser.open(url)

    time.sleep(2)
    keyboard.press_and_release('ctrl+w')
    

: 

In [12]:
!pip install keyboard

  Using cached keyboard-0.13.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached pyobjc-11.1-py3-none-any.whl.metadata (25 kB)
  Using cached pyobjc-11.1-py3-none-any.whl.metadata (25 kB)
  Using cached pyobjc_framework_adservices-11.1-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached pyobjc_framework_adservices-11.1-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached pyobjc_framework_adsupport-11.1-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached pyobjc_framework_apptrackingtransparency-11.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pyobjc_framework_adsupport-11.1-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached pyobjc_framework_apptrackingtransparency-11.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pyobjc_framework_accounts-11.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pyobjc_framework_accounts-11.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pyobjc_framework_applescriptkit-11.1-py2.py3-none-any.whl.metadata (2.5 kB)
  Usi

In [4]:
import requests
from bs4 import BeautifulSoup
import os
import time
import csv
import re
from pathlib import Path

# --- Configuration ---
# The range of schematic IDs you want to download.
# The original script started around 1824 and the site has over 19,000.
# Start with a small range for testing, e.g., (1824, 1834).
START_ID = 1824
END_ID = 19000 

# Get the project root directory (two levels up from notebooks/)
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"

# Folder to save the downloaded schematic files (inside data/)
OUTPUT_FOLDER = DATA_DIR / "schematics"

# Name for the CSV file that will store the metadata (inside data/)
METADATA_FILE = DATA_DIR / "schematics_metadata.csv"

# Delay between requests in seconds to be respectful to the server.
REQUEST_DELAY = 1.5

# Login credentials - FILL THESE IN
LOGIN_URL = "https://www.minecraft-schematics.com/login/"  # Adjust if needed
USERNAME = "mous0082@gmail.com"  # Your username
PASSWORD = "mine1000Fryd"  # Your password

# --- End of Configuration ---

def login_to_site(session):
    """
    Login to the minecraft-schematics.com website.
    
    Args:
        session: A requests.Session object
        
    Returns:
        True if login successful, False otherwise
    """
    if not USERNAME or not PASSWORD:
        print("WARNING: No login credentials provided. Downloads may fail.")
        print("Please set USERNAME and PASSWORD in the configuration section.")
        return False
    
    try:
        # First, get the login page to retrieve CSRF token if needed
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = session.get(LOGIN_URL, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Look for CSRF token (common in Django/web forms)
        csrf_token = None
        csrf_input = soup.find('input', {'name': 'csrfmiddlewaretoken'})
        if csrf_input:
            csrf_token = csrf_input.get('value')
        
        # Prepare login data
        login_data = {
            'username': USERNAME,
            'password': PASSWORD,
        }
        
        if csrf_token:
            login_data['csrfmiddlewaretoken'] = csrf_token
        
        # Attempt login
        login_response = session.post(LOGIN_URL, data=login_data, headers=headers, timeout=10)
        
        # Check if login was successful (adjust based on actual site behavior)
        if login_response.status_code == 200 and 'logout' in login_response.text.lower():
            print("✓ Successfully logged in!")
            return True
        else:
            print("✗ Login failed. Please check your credentials.")
            return False
            
    except Exception as e:
        print(f"Error during login: {e}")
        return False

def get_schematic_metadata(soup):
    """
    Parses the BeautifulSoup object of a schematic page to extract metadata.
    
    Args:
        soup: A BeautifulSoup object representing the parsed HTML of the page.
        
    Returns:
        A dictionary containing the schematic's metadata, or None if not found.
    """
    metadata = {}
    
    # The metadata is stored in a table. We find the table rows.
    # We look for the 'strong' tag (e.g., <strong>Category</strong>) and then get the
    # text from the next sibling element in the table.
    
    def get_info_from_table(label):
        """Helper function to find a label in the table and return its value."""
        label_tag = soup.find('strong', string=re.compile(f'^{label}'))
        if label_tag and label_tag.parent and label_tag.parent.next_sibling:
            # The value is in the next table cell (td)
            value_cell = label_tag.parent.find_next_sibling('td')
            if value_cell:
                return value_cell.get_text(strip=True)
        return 'N/A'

    metadata['title'] = soup.find('h1').get_text(strip=True) if soup.find('h1') else 'N/A'
    metadata['category'] = get_info_from_table('Category')
    metadata['theme'] = get_info_from_table('Theme')
    metadata['size'] = get_info_from_table('Size')
    metadata['submitted_by'] = get_info_from_table('Submitted by')
    metadata['posted_on'] = get_info_from_table('Posted on')
    
    # Extract the description
    # Description is typically in a div or paragraph after the metadata table
    description = 'N/A'
    description_div = soup.find('div', class_=re.compile('description|content|detail', re.I))
    if description_div:
        description = description_div.get_text(strip=True)
    else:
        # Try finding any paragraph that might contain the description
        paragraphs = soup.find_all('p')
        if paragraphs:
            # Take the first substantial paragraph (more than 20 chars)
            for p in paragraphs:
                text = p.get_text(strip=True)
                if len(text) > 20:
                    description = text
                    break
    metadata['description'] = description
    
    # Extract the number of downloads
    downloads_text = get_info_from_table('Download\\(s\\)')
    download_count = '0'
    if downloads_text and 'Has been downloaded' in downloads_text:
        # Use regex to find the number in the string
        match = re.search(r'(\d+)', downloads_text)
        if match:
            download_count = match.group(1)
    metadata['downloads'] = download_count
    
    return metadata

In [5]:
def main():
    """
    Main function to run the scraper.
    """
    # Create the output directory if it doesn't exist
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
    print(f"Created directory: {OUTPUT_FOLDER}")

    # Create a session to maintain cookies/authentication
    session = requests.Session()
    
    # Login to the site
    print("\nAttempting to login...")
    if not login_to_site(session):
        print("\nContinuing without login. Downloads may fail if authentication is required.")
        response = input("Continue anyway? (y/n): ")
        if response.lower() != 'y':
            print("Scraping cancelled.")
            return

    # Prepare to write metadata to a CSV file
    csv_headers = ['id', 'title', 'category', 'theme', 'size', 'submitted_by', 'posted_on', 'downloads', 'description', 'local_filename', 'status']
    
    print(f"\nStarting scrape. Metadata will be saved to '{METADATA_FILE}'")

    with open(METADATA_FILE, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_headers)
        writer.writeheader()

        # Loop through the specified range of schematic IDs
        for schematic_id in range(START_ID, END_ID + 1):
            page_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/"
            download_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/download/"  # Corrected URL

            print(f"\nProcessing Schematic ID: {schematic_id}")
            print(f"Page URL: {page_url}")

            try:
                # --- Step 1: Scrape the metadata from the schematic page ---
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                    'Referer': page_url
                }
                response = session.get(page_url, headers=headers, timeout=10)
                
                # Check if the page exists
                if response.status_code == 404:
                    print(f"  - Schematic not found (404). Skipping.")
                    metadata = {
                        'id': schematic_id,
                        'title': 'N/A',
                        'category': 'N/A',
                        'theme': 'N/A',
                        'size': 'N/A',
                        'submitted_by': 'N/A',
                        'posted_on': 'N/A',
                        'downloads': '0',
                        'description': 'N/A',
                        'local_filename': 'N/A',
                        'status': 'not_found'
                    }
                    writer.writerow(metadata)
                    continue
                elif response.status_code != 200:
                    print(f"Failed to retrieve page. Status code: {response.status_code}")
                    continue

                # Parse the HTML content
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Extract metadata
                metadata = get_schematic_metadata(soup)
                if not metadata:
                    print("Could not find metadata table on the page.")
                    continue
                
                metadata['id'] = schematic_id
                print(f"  - Title: {metadata['title']}")
                print(f"  - Category: {metadata['category']}")
                print(f"  - Downloads: {metadata['downloads']}")
                # Show truncated description in output, but save full description to CSV
                desc_preview = metadata['description'][:50] + "..." if len(metadata['description']) > 50 else metadata['description']
                print(f"  - Description: {desc_preview}")

                # --- Step 2: Download the schematic file ---
                print(f"Downloading from: {download_url}")
                download_response = session.get(download_url, headers=headers, timeout=15, allow_redirects=True)

                # Check if we got HTML instead of a schematic file
                content_type = download_response.headers.get('content-type', '')
                
                if download_response.status_code == 200 and 'text/html' not in content_type:
                    # Check content disposition for the real filename, fallback to ID
                    content_disp = download_response.headers.get('content-disposition')
                    if content_disp and 'filename=' in content_disp:
                        filename_match = re.findall('filename="(.+)"', content_disp)
                        filename = filename_match[0] if filename_match else f"{schematic_id}.schematic"
                    else:
                        filename = f"{schematic_id}.schematic"
                    
                    filepath = OUTPUT_FOLDER / filename
                    
                    # Save the file
                    with open(filepath, 'wb') as f:
                        f.write(download_response.content)
                    
                    print(f"  - Successfully saved to {filepath} ({len(download_response.content)} bytes)")
                    metadata['local_filename'] = filename
                    metadata['status'] = 'downloaded'
                    
                    # Write the collected data to the CSV file (with full description)
                    writer.writerow(metadata)
                elif 'text/html' in content_type:
                    print(f"  - Download returned HTML instead of schematic file. File may not be available.")
                    metadata['local_filename'] = 'N/A'
                    metadata['status'] = 'html_response'
                    writer.writerow(metadata)
                else:
                    print(f"  - Failed to download file. Status code: {download_response.status_code}")
                    metadata['local_filename'] = 'N/A'
                    metadata['status'] = f'error_{download_response.status_code}'
                    writer.writerow(metadata)

            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                metadata = {
                    'id': schematic_id,
                    'title': 'N/A',
                    'category': 'N/A',
                    'theme': 'N/A',
                    'size': 'N/A',
                    'submitted_by': 'N/A',
                    'posted_on': 'N/A',
                    'downloads': '0',
                    'description': 'N/A',
                    'local_filename': 'N/A',
                    'status': 'request_error'
                }
                writer.writerow(metadata)
            
            # Be polite and wait before the next request
            time.sleep(REQUEST_DELAY)

    print(f"\nScraping complete. All data saved.")

# Testing Login and Authentication

Let's test the login process step by step to understand how the website handles authentication.

In [10]:
# Step 1: Let's first examine the login page structure
import requests
from bs4 import BeautifulSoup

LOGIN_URL = "https://www.minecraft-schematics.com/login/"

print("=" * 60)
print("STEP 1: Fetching the login page")
print("=" * 60)

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

response = session.get(LOGIN_URL, headers=headers)
print(f"Status Code: {response.status_code}")
print(f"URL after redirects: {response.url}")
print(f"\nCookies received: {dict(session.cookies)}")

# Parse the page
soup = BeautifulSoup(response.content, 'html.parser')

# Look for the login form
print("\n" + "=" * 60)
print("STEP 2: Analyzing the login form")
print("=" * 60)

forms = soup.find_all('form')
print(f"Number of forms found: {len(forms)}")

for i, form in enumerate(forms):
    print(f"\n--- Form {i+1} ---")
    print(f"Action: {form.get('action')}")
    print(f"Method: {form.get('method')}")
    
    # Find all input fields
    inputs = form.find_all('input')
    print(f"Input fields ({len(inputs)}):")
    for inp in inputs:
        print(f"  - Name: {inp.get('name')}, Type: {inp.get('type')}, Value: {inp.get('value')}")

print("\n" + "=" * 60)
print("STEP 3: Looking for CSRF tokens")
print("=" * 60)

# Look for common CSRF token patterns
csrf_patterns = ['csrf', 'token', '_token', 'authenticity_token']
for pattern in csrf_patterns:
    csrf_input = soup.find('input', {'name': lambda x: x and pattern in x.lower()})
    if csrf_input:
        print(f"Found potential CSRF field: {csrf_input.get('name')} = {csrf_input.get('value')[:20]}...")
        
# Also check meta tags
meta_csrf = soup.find('meta', {'name': lambda x: x and 'csrf' in x.lower() if x else False})
if meta_csrf:
    print(f"Found CSRF in meta tag: {meta_csrf}")

STEP 1: Fetching the login page
Status Code: 200
URL after redirects: https://www.minecraft-schematics.com/login/

Cookies received: {'PHPSESSID': 'c80cc9abfa27e21b30502def822b27e8'}

STEP 2: Analyzing the login form
Number of forms found: 1

--- Form 1 ---
Action: /login/action/
Method: post
Input fields (4):
  - Name: email_state, Type: hidden, Value: error
  - Name: password_state, Type: hidden, Value: error
  - Name: email, Type: text, Value: 
  - Name: password, Type: password, Value: None

STEP 3: Looking for CSRF tokens


In [11]:
# Step 2: Try to perform login with detailed logging
print("=" * 60)
print("STEP 4: Attempting login")
print("=" * 60)

# Get fresh session and login page
session2 = requests.Session()
response = session2.get(LOGIN_URL, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Get CSRF token if present
csrf_token = None
csrf_input = soup.find('input', {'name': lambda x: x and 'csrf' in x.lower() if x else False})
if csrf_input:
    csrf_token = csrf_input.get('value')
    print(f"CSRF token found: {csrf_token[:20]}...")

# Prepare login data
login_data = {
    'username': 'mous0082@gmail.com',  # Your username/email
    'password': 'mine1000Fryd',  # Your password
}

if csrf_token:
    login_data['csrfmiddlewaretoken'] = csrf_token

print(f"\nLogin data keys: {list(login_data.keys())}")
print(f"Cookies before login: {dict(session2.cookies)}")

# Attempt login
print("\nSending POST request...")
login_response = session2.post(LOGIN_URL, data=login_data, headers=headers)

print(f"\nStatus Code: {login_response.status_code}")
print(f"Final URL: {login_response.url}")
print(f"Cookies after login: {dict(session2.cookies)}")

# Check response content for clues
print("\n" + "=" * 60)
print("STEP 5: Analyzing login response")
print("=" * 60)

soup_response = BeautifulSoup(login_response.content, 'html.parser')

# Look for success/error messages
for keyword in ['success', 'welcome', 'logout', 'error', 'invalid', 'incorrect']:
    if keyword in login_response.text.lower():
        print(f"Found keyword '{keyword}' in response")

# Check if there's a logout link (indicates successful login)
logout_link = soup_response.find('a', href=lambda x: x and 'logout' in x.lower() if x else False)
if logout_link:
    print(f"\n✓ SUCCESS! Found logout link: {logout_link.get('href')}")
else:
    print("\n✗ No logout link found - login may have failed")

# Check for error messages
error_div = soup_response.find('div', class_=lambda x: x and 'error' in x.lower() if x else False)
if error_div:
    print(f"Error message: {error_div.get_text(strip=True)}")

STEP 4: Attempting login

Login data keys: ['username', 'password']
Cookies before login: {'PHPSESSID': 'ee2f44b6fb8ebd86cecc7841374ca091'}

Sending POST request...

Status Code: 200
Final URL: https://www.minecraft-schematics.com/login/
Cookies after login: {'PHPSESSID': 'ee2f44b6fb8ebd86cecc7841374ca091'}

STEP 5: Analyzing login response
Found keyword 'error' in response

✗ No logout link found - login may have failed


In [12]:
# Step 3: Test if we can access a download with the logged-in session
print("=" * 60)
print("STEP 6: Testing download with authenticated session")
print("=" * 60)

# Try to download a known schematic
test_id = 1825  # Change this to a known valid ID
download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/action/?type=schematic"
page_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/"

print(f"Test schematic ID: {test_id}")
print(f"Page URL: {page_url}")
print(f"Download URL: {download_url}")

# First check the page
print("\nFetching schematic page...")
page_response = session2.get(page_url, headers=headers)
print(f"Page status: {page_response.status_code}")

# Then try download
print("\nAttempting download...")
download_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Referer': page_url
}
download_response = session2.get(download_url, headers=download_headers, allow_redirects=True)

print(f"Download status: {download_response.status_code}")
print(f"Content-Type: {download_response.headers.get('content-type')}")
print(f"Content-Length: {len(download_response.content)} bytes")
print(f"Content-Disposition: {download_response.headers.get('content-disposition')}")

# Check if we got HTML or binary data
if 'text/html' in download_response.headers.get('content-type', ''):
    print("\n✗ Received HTML (not a file)")
    # Show first 500 chars of response
    print(f"Response preview:\n{download_response.text[:500]}")
else:
    print("\n✓ Received binary data (likely a schematic file!)")
    # Show first few bytes in hex
    print(f"First 20 bytes (hex): {download_response.content[:20].hex()}")
    print(f"First 20 bytes (repr): {repr(download_response.content[:20])}")

STEP 6: Testing download with authenticated session
Test schematic ID: 1825
Page URL: https://www.minecraft-schematics.com/schematic/1825/
Download URL: https://www.minecraft-schematics.com/schematic/1825/download/action/?type=schematic

Fetching schematic page...
Page status: 200

Attempting download...
Download status: 200
Content-Type: text/html; charset=UTF-8
Content-Length: 22860 bytes
Content-Disposition: None

✗ Received HTML (not a file)
Response preview:
<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="utf-8">
	<title>Minecraft Schematics, the Minecraft creations and schematics reference. Minecraft Worlds, minecraft maps and minecraft schematics.</title>
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<meta name="description" content="Minecraft Schematics is the best place to find Minecraft creations, schematics, maps and worlds to download. Browse, share, download, comment, add to favorites..." />	<meta name="keywo


In [13]:
# Fixed login test based on the actual form structure
print("=" * 60)
print("CORRECTED LOGIN ATTEMPT")
print("=" * 60)

session3 = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# First get the login page to establish session
print("Getting login page...")
session3.get(LOGIN_URL, headers=headers)
print(f"Session cookies: {dict(session3.cookies)}")

# Prepare correct login data based on actual form fields
login_data = {
    'email': 'mous0082@gmail.com',  # Use 'email' not 'username'
    'password': 'mine1000Fryd',
}

print(f"\nPosting to: https://www.minecraft-schematics.com/login/action/")
print(f"Login data: {list(login_data.keys())}")

# Post to the correct action URL
login_response = session3.post(
    'https://www.minecraft-schematics.com/login/action/',
    data=login_data,
    headers=headers,
    allow_redirects=True
)

print(f"\nStatus: {login_response.status_code}")
print(f"Final URL: {login_response.url}")
print(f"Cookies: {dict(session3.cookies)}")

# Check for success indicators
soup_resp = BeautifulSoup(login_response.content, 'html.parser')
logout_link = soup_resp.find('a', href=lambda x: x and 'logout' in x.lower() if x else False)

if logout_link:
    print("\n✓ SUCCESS! Found logout link - we are logged in!")
else:
    print("\n✗ No logout link found")
    # Check for error keywords
    if 'error' in login_response.text.lower():
        print("Found 'error' in response")
    if 'invalid' in login_response.text.lower():
        print("Found 'invalid' in response")

# Now test download with this session
print("\n" + "=" * 60)
print("TESTING DOWNLOAD WITH CORRECTED SESSION")
print("=" * 60)

test_id = 1825
download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/action/?type=schematic"
page_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/"

download_response = session3.get(download_url, headers={'User-Agent': headers['User-Agent'], 'Referer': page_url})

print(f"Download status: {download_response.status_code}")
print(f"Content-Type: {download_response.headers.get('content-type')}")
print(f"Content-Length: {len(download_response.content)} bytes")

if 'text/html' in download_response.headers.get('content-type', ''):
    print("\n✗ Still getting HTML")
else:
    print("\n✓ Got binary data!")
    print(f"First 20 bytes: {download_response.content[:20].hex()}")

CORRECTED LOGIN ATTEMPT
Getting login page...
Session cookies: {'PHPSESSID': '33dd2f161f8d96df1b8d7cacf56f68e2'}

Posting to: https://www.minecraft-schematics.com/login/action/
Login data: ['email', 'password']

Status: 200
Final URL: https://www.minecraft-schematics.com/login/?error=captcha
Cookies: {'PHPSESSID': '33dd2f161f8d96df1b8d7cacf56f68e2'}

✗ No logout link found
Found 'error' in response

TESTING DOWNLOAD WITH CORRECTED SESSION
Download status: 200
Content-Type: text/html; charset=UTF-8
Content-Length: 22860 bytes

✗ Still getting HTML


## Solution: Use Browser Cookies

Since the site requires CAPTCHA for login, we'll use cookies from your browser session where you're already logged in.

### Steps:
1. Open your browser and make sure you're logged into minecraft-schematics.com
2. Get your cookies (instructions below)
3. Use them in the script

In [15]:
# Method 1: Manually copy your browser cookies
# 1. Go to minecraft-schematics.com in your browser while logged in
# 2. Open Developer Tools (F12)
# 3. Go to Console tab
# 4. Type: document.cookie
# 5. Copy the result and paste it here

# Example: "PHPSESSID=abc123; user_id=456; session_token=xyz..."
BROWSER_COOKIES_STRING = "PHPSESSID=ef4869c1d3914a0bfe4b694fec0a3725; uid=2578062; ukey=q1nbs48czsf8deca7newg6fa1m6urr45atnpwto3da1j0; utoken=kdey4etck4ewwlezsgth"  # Paste your cookies here

# Parse the cookie string
def parse_cookie_string(cookie_str):
    """Parse a browser cookie string into a dictionary"""
    cookies = {}
    if cookie_str:
        for item in cookie_str.split(';'):
            item = item.strip()
            if '=' in item:
                key, value = item.split('=', 1)
                cookies[key] = value
    return cookies

# Test with browser cookies
if BROWSER_COOKIES_STRING:
    print("Testing with browser cookies...")
    session_browser = requests.Session()
    
    # Add cookies to session
    cookies_dict = parse_cookie_string(BROWSER_COOKIES_STRING)
    for key, value in cookies_dict.items():
        session_browser.cookies.set(key, value)
    
    print(f"Cookies loaded: {list(cookies_dict.keys())}")
    
    # Test download
    test_id = 1857
    download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/action/?type=schematic"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    download_response = session_browser.get(download_url, headers=headers)
    
    print(f"Download status: {download_response.status_code}")
    print(f"Content-Type: {download_response.headers.get('content-type')}")
    print(f"Content-Length: {len(download_response.content)} bytes")
    
    if 'text/html' not in download_response.headers.get('content-type', ''):
        print("\n✓ SUCCESS! Got binary data with browser cookies!")
    else:
        print("\n✗ Still getting HTML - cookies may have expired")
else:
    print("Please set BROWSER_COOKIES_STRING with your browser cookies")

Testing with browser cookies...
Cookies loaded: ['PHPSESSID', 'uid', 'ukey', 'utoken']
Download status: 200
Content-Type: text/html; charset=UTF-8
Content-Length: 22468 bytes

✗ Still getting HTML - cookies may have expired


In [16]:
# Let's investigate the actual download behavior more carefully
print("=" * 60)
print("INVESTIGATING DOWNLOAD MECHANISM")
print("=" * 60)

session_investigate = requests.Session()
cookies_dict = parse_cookie_string(BROWSER_COOKIES_STRING)
for key, value in cookies_dict.items():
    session_investigate.cookies.set(key, value)

test_id = 1825
page_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
}

# First, get the schematic page
print(f"1. Fetching page: {page_url}")
page_response = session_investigate.get(page_url, headers=headers)
print(f"   Status: {page_response.status_code}")

# Parse the page to look for download links
soup = BeautifulSoup(page_response.content, 'html.parser')

# Find all links that might be download links
print("\n2. Looking for download links on the page...")
download_links = []
for link in soup.find_all('a'):
    href = link.get('href', '')
    text = link.get_text(strip=True).lower()
    if 'download' in href.lower() or 'download' in text:
        download_links.append({
            'href': href,
            'text': text,
            'full_url': href if href.startswith('http') else f"https://www.minecraft-schematics.com{href}"
        })
        print(f"   Found: {link.get_text(strip=True)[:50]}")
        print(f"   URL: {href}")
        print()

# Try each download link
print("\n3. Testing each download link...")
for i, link_info in enumerate(download_links[:3], 1):  # Test first 3 links
    print(f"\n--- Testing Link {i} ---")
    print(f"URL: {link_info['full_url']}")
    
    headers['Referer'] = page_url
    test_response = session_investigate.get(link_info['full_url'], headers=headers, allow_redirects=True)
    
    print(f"Status: {test_response.status_code}")
    print(f"Final URL: {test_response.url}")
    print(f"Content-Type: {test_response.headers.get('content-type')}")
    print(f"Content-Length: {len(test_response.content)} bytes")
    print(f"Content-Disposition: {test_response.headers.get('content-disposition')}")
    
    if 'text/html' not in test_response.headers.get('content-type', ''):
        print("✓ This link returned binary data!")
        print(f"First 20 bytes: {test_response.content[:20].hex()}")
        break
    else:
        print("✗ This link returned HTML")

INVESTIGATING DOWNLOAD MECHANISM
1. Fetching page: https://www.minecraft-schematics.com/schematic/1825/
   Status: 200

2. Looking for download links on the page...
   Found: Most Downloaded
   URL: /most-downloaded/

   Found: Details & Download »
   URL: /schematic/26533/

   Found: Details & Download »
   URL: /schematic/26532/

   Found: Details & Download »
   URL: /schematic/26531/

   Found: Details & Download »
   URL: /schematic/26530/

   Found: Details & Download »
   URL: /schematic/26527/

   Found: Details & Download »
   URL: /schematic/26522/

   Found: Details & Download »
   URL: /schematic/26521/

   Found: Details & Download »
   URL: /schematic/26519/

   Found: Details & Download »
   URL: /schematic/26518/

   Found: See the full list »
   URL: /most-downloaded/

   Found: Details & Download »
   URL: /schematic/235/

   Found: Details & Download »
   URL: /schematic/270/

   Found: Details & Download »
   URL: /schematic/218/

   Found: Most Downloaded
   URL: /

In [17]:
# Let's try with ID 1857 which has content
print("=" * 60)
print("CHECKING SCHEMATIC ID 1857")
print("=" * 60)

session_test = requests.Session()
cookies_dict = parse_cookie_string(BROWSER_COOKIES_STRING)
for key, value in cookies_dict.items():
    session_test.cookies.set(key, value)

test_id = 1857
page_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

print(f"Fetching page: {page_url}")
page_response = session_test.get(page_url, headers=headers)
print(f"Status: {page_response.status_code}")

soup = BeautifulSoup(page_response.content, 'html.parser')

# Look specifically for download buttons/links
print("\nLooking for download-related elements...")
found_download = False
for elem in soup.find_all(['a', 'button']):
    text = elem.get_text(strip=True).lower()
    href = elem.get('href', '')
    onclick = elem.get('onclick', '')
    
    if any(word in text for word in ['schematic', 'download', '.schem']) or \
       any(word in href for word in ['download', 'action', 'schematic']) or \
       'download' in onclick.lower():
        print(f"\nElement: {elem.name}")
        print(f"  Text: {text[:60]}")
        print(f"  Href: {href}")
        if onclick:
            print(f"  Onclick: {onclick[:60]}")
        found_download = True

if not found_download:
    print("No obvious download elements found")

# Try the original download URL format
print("\n" + "=" * 60)
print("TRYING ORIGINAL DOWNLOAD URL FORMAT")
print("=" * 60)

download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/action/?type=schematic"
print(f"URL: {download_url}")

headers['Referer'] = page_url
response = session_test.get(download_url, headers=headers, allow_redirects=True)

print(f"\nStatus: {response.status_code}")
print(f"Final URL: {response.url}")
print(f"Content-Type: {response.headers.get('content-type')}")
print(f"Content-Length: {len(response.content)}")

# Check if it's HTML or binary
if 'text/html' in response.headers.get('content-type', ''):
    print("\n✗ Received HTML")
    # Look for error messages in the HTML
    error_soup = BeautifulSoup(response.content, 'html.parser')
    title = error_soup.find('title')
    if title:
        print(f"Page title: {title.get_text()}")
    # Check for any error/login messages
    if 'login' in response.text.lower():
        print("⚠ Page mentions 'login' - authentication may be required")
    if 'permission' in response.text.lower():
        print("⚠ Page mentions 'permission' - access may be restricted")
else:
    print("\n✓ Received binary data!")
    print(f"First 20 bytes (hex): {response.content[:20].hex()}")
    print(f"First 20 bytes (repr): {repr(response.content[:20])}")

CHECKING SCHEMATIC ID 1857
Fetching page: https://www.minecraft-schematics.com/schematic/1857/
Status: 200

Looking for download-related elements...

Element: a
  Text: minecraft schematics
  Href: /

Element: a
  Text: most downloaded
  Href: /most-downloaded/

Element: a
  Text: post your creation
  Href: /schematic/add/

Element: a
  Text: 
  Href: https://www.minecraft-schematics.com/schematics/pictures/1857/large-picture-1857.jpg?time=1384700274

Element: a
  Text: download
  Href: /schematic/1857/download/

Element: a
  Text: 
  Href: /schematic/1866/

Element: a
  Text: 
  Href: /schematic/5988/

Element: a
  Text: 
  Href: /schematic/5989/

Element: a
  Text: 
  Href: /schematic/5987/

Element: a
  Text: 
  Href: https://www.minecraft-schematics.com/upload/pictures/1857/large-picture-1857-1.jpg?time=1384700274

Element: a
  Text: 
  Href: https://www.minecraft-schematics.com/upload/pictures/1857/large-picture-1857-2.png?time=1384700274

Element: a
  Text: 
  Href: https://www.m

In [18]:
# Test the CORRECT download URL we found on the page
print("=" * 60)
print("TESTING CORRECT DOWNLOAD URL")
print("=" * 60)

correct_download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/"
print(f"Correct URL from page: {correct_download_url}")

response2 = session_test.get(correct_download_url, headers=headers, allow_redirects=True)

print(f"\nStatus: {response2.status_code}")
print(f"Final URL: {response2.url}")
print(f"Content-Type: {response2.headers.get('content-type')}")
print(f"Content-Disposition: {response2.headers.get('content-disposition')}")
print(f"Content-Length: {len(response2.content)}")

if 'text/html' in response2.headers.get('content-type', ''):
    print("\n✗ Still HTML")
    if 'login' in response2.text.lower():
        print("⚠ Mentions 'login'")
else:
    print("\n✓✓✓ GOT BINARY DATA!")
    print(f"First 20 bytes (hex): {response2.content[:20].hex()}")
    print(f"First 20 bytes (repr): {repr(response2.content[:20])}")
    
    # Save a test file
    test_file = Path("/Users/moustholmes/minecraft_voxel_flow/data/test_download.schematic")
    with open(test_file, 'wb') as f:
        f.write(response2.content)
    print(f"\n✓ Saved test file to: {test_file}")

TESTING CORRECT DOWNLOAD URL
Correct URL from page: https://www.minecraft-schematics.com/schematic/1857/download/

Status: 200
Final URL: https://www.minecraft-schematics.com/login/
Content-Type: text/html; charset=UTF-8
Content-Disposition: None
Content-Length: 13782

✗ Still HTML
⚠ Mentions 'login'


In [19]:
# The cookies are being redirected to login. 
# This means either:
# 1. The cookies expired
# 2. The cookies are domain-specific and not being sent correctly
# 3. Additional cookies/headers are needed

print("=" * 60)
print("COOKIE DOMAIN INVESTIGATION")
print("=" * 60)

# Try setting cookies with proper domain
session_domain = requests.Session()

for key, value in cookies_dict.items():
    session_domain.cookies.set(
        key, 
        value, 
        domain='.minecraft-schematics.com',
        path='/'
    )

print("Cookies set with domain '.minecraft-schematics.com':")
print(dict(session_domain.cookies))

# Add more browser-like headers
browser_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Referer': page_url,
}

print(f"\nTrying download with domain cookies...")
response3 = session_domain.get(correct_download_url, headers=browser_headers, allow_redirects=False)

print(f"Status: {response3.status_code}")
print(f"Headers: {dict(response3.headers)}")

if response3.status_code in [301, 302, 303, 307, 308]:
    print(f"Redirect to: {response3.headers.get('Location')}")
    
if 'text/html' not in response3.headers.get('content-type', ''):
    print("\n✓✓✓ SUCCESS! Got binary data!")
else:
    print("\n✗ Still getting HTML/redirect")
    print("\n⚠ Your browser cookies appear to be session-specific.")
    print("   You may need to use browser automation (like Selenium) to maintain the session.")

COOKIE DOMAIN INVESTIGATION
Cookies set with domain '.minecraft-schematics.com':
{'PHPSESSID': 'ef4869c1d3914a0bfe4b694fec0a3725', 'uid': '2578062', 'ukey': 'q1nbs48czsf8deca7newg6fa1m6urr45atnpwto3da1j0', 'utoken': 'kdey4etck4ewwlezsgth'}

Trying download with domain cookies...
Status: 302
Headers: {'Date': 'Tue, 07 Oct 2025 10:42:31 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'pragma': 'no-cache', 'Nel': '{"report_to":"cf-nel","success_fraction":0.0,"max_age":604800}', 'location': '/login/', 'x-powered-by': 'PleskLin', 'cf-cache-status': 'DYNAMIC', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains', 'Server-Timing': 'cfCacheStatus;desc="DYNAMIC", cfEdge;dur=8,cfOrigin;dur=139', 'Report-To': '{"group":"cf-nel","max_age":604800,"endpoints":[{"url":"https://a.nel.cloudflare.com/report

## Final Solution: Use Selenium for Browser Automation

Since:
1. The site requires CAPTCHA for login (can't automate login)
2. Cookies expire/are invalidated when copied
3. The download requires an authenticated session

**We need to use Selenium** to control your actual browser where you're already logged in.

In [20]:
# Install selenium
!pip install selenium webdriver-manager

  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.9 MB/s  0:00:02 eta 0:00:01
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [selenium]/12 [selenium]


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Test Selenium setup
print("Setting up Chrome driver...")
options = webdriver.ChromeOptions()
# Use your existing Chrome profile to maintain login session
# Uncomment and adjust the path if you want to use your actual Chrome profile
# options.add_argument("user-data-dir=/Users/moustholmes/Library/Application Support/Google/Chrome")
# options.add_argument("profile-directory=Default")

# For now, start fresh - you'll need to login manually once
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

print("✓ Chrome driver initialized")
print("\nOpening minecraft-schematics.com...")
driver.get("https://www.minecraft-schematics.com/")

print("\n" + "=" * 60)
print("MANUAL STEP REQUIRED")
print("=" * 60)
print("1. A Chrome window has opened")
print("2. Please LOGIN to minecraft-schematics.com in that window")
print("3. Once logged in, come back here and run the next cell")
print("=" * 60)

Setting up Chrome driver...
✓ Chrome driver initialized

Opening minecraft-schematics.com...

MANUAL STEP REQUIRED
1. A Chrome window has opened
2. Please LOGIN to minecraft-schematics.com in that window
3. Once logged in, come back here and run the next cell


In [ ]:
# Close the previous driver first
try:
    driver.quit()
    print("Closed previous driver")
except:
    pass

# Alternative: Use your existing Chrome profile where you're already logged in
import os

# Find your Chrome user data directory
chrome_user_data = os.path.expanduser("~/Library/Application Support/Google/Chrome")

print("Setting up Chrome with your existing profile...")
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={chrome_user_data}")
options.add_argument("profile-directory=Default")  # Use Default profile, or change to "Profile 1", etc.

# Important: Close all Chrome windows before running this!
print("\n" + "="*60)
print("IMPORTANT: Close ALL Chrome windows first!")
print("="*60)
input("Press Enter after closing all Chrome windows...")

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    print("\n✓ Connected to your Chrome profile")
    print("✓ Opening minecraft-schematics.com...")
    driver.get("https://www.minecraft-schematics.com/")
    time.sleep(2)
    
    # Check if already logged in
    if 'logout' in driver.page_source.lower():
        print("\n✓✓✓ You are already logged in!")
        print("Ready to proceed with scraping!")
    else:
        print("\n⚠ Not logged in. Please login in the browser window that just opened.")
        
except Exception as e:
    print(f"\n✗ Error: {e}")
    print("\nIf you get a 'Chrome is already running' error:")
    print("1. Close ALL Chrome windows (including minimized ones)")
    print("2. Wait a few seconds")
    print("3. Run this cell again")

In [22]:
# Install Firefox driver
!pip install webdriver-manager

In [23]:
# Close any existing driver
try:
    driver.quit()
    print("Closed previous driver")
except:
    pass

# Use Firefox with your existing profile
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

print("Setting up Firefox with your existing profile...")

# Find your Firefox profile path
firefox_profile_path = os.path.expanduser("~/Library/Application Support/Firefox/Profiles")

# List available profiles
if os.path.exists(firefox_profile_path):
    profiles = [d for d in os.listdir(firefox_profile_path) if os.path.isdir(os.path.join(firefox_profile_path, d))]
    print(f"\nFound Firefox profiles: {profiles}")
    if profiles:
        # Use the first profile (usually the default one)
        profile_dir = os.path.join(firefox_profile_path, profiles[0])
        print(f"Using profile: {profiles[0]}")
    else:
        print("No profiles found")
        profile_dir = None
else:
    print("Firefox profile directory not found")
    profile_dir = None

options = webdriver.FirefoxOptions()
if profile_dir:
    options.add_argument("-profile")
    options.add_argument(profile_dir)

print("\n" + "="*60)
print("IMPORTANT: Close ALL Firefox windows first!")
print("="*60)
input("Press Enter after closing all Firefox windows...")

try:
    driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
    print("\n✓ Connected to your Firefox profile")
    print("✓ Opening minecraft-schematics.com...")
    driver.get("https://www.minecraft-schematics.com/")
    time.sleep(2)
    
    # Check if already logged in
    if 'logout' in driver.page_source.lower():
        print("\n✓✓✓ You are already logged in!")
        print("Ready to proceed with scraping!")
    else:
        print("\n⚠ Not logged in. Please login in the browser window that just opened.")
        
except Exception as e:
    print(f"\n✗ Error: {e}")
    print("\nIf you get an error:")
    print("1. Close ALL Firefox windows")
    print("2. Wait a few seconds")
    print("3. Run this cell again")

Closed previous driver
Setting up Firefox with your existing profile...

Found Firefox profiles: ['rbs0inu4.default', 'gjtt4crb.default-release']
Using profile: rbs0inu4.default

IMPORTANT: Close ALL Firefox windows first!

✗ Error: Message: Process unexpectedly closed with status 0


If you get an error:
1. Close ALL Firefox windows
2. Wait a few seconds
3. Run this cell again


In [24]:
# Simpler approach: Start fresh Firefox and manually add your cookies
print("Starting fresh Firefox instance...")

options = webdriver.FirefoxOptions()
# Don't use existing profile to avoid conflicts

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
print("✓ Firefox started")

# Go to the site first (required before adding cookies)
print("Opening minecraft-schematics.com...")
driver.get("https://www.minecraft-schematics.com/")
time.sleep(2)

# Now add your cookies from earlier
print("\nAdding your login cookies...")
cookies_dict = {
    'PHPSESSID': 'ef4869c1d3914a0bfe4b694fec0a3725',
    'uid': '2578062',
    'ukey': 'q1nbs48czsf8deca7newg6fa1m6urr45atnpwto3da1j0',
    'utoken': 'kdey4etck4ewwlezsgth'
}

for name, value in cookies_dict.items():
    driver.add_cookie({'name': name, 'value': value, 'domain': '.minecraft-schematics.com'})
    print(f"  Added cookie: {name}")

# Refresh to apply cookies
print("\nRefreshing page to apply cookies...")
driver.refresh()
time.sleep(2)

# Check if logged in
if 'logout' in driver.page_source.lower():
    print("\n✓✓✓ Successfully logged in with cookies!")
    print("Ready to start scraping!")
else:
    print("\n⚠ Cookies didn't work. The session may have expired.")
    print("Please try getting fresh cookies from Firefox:")
    print("1. In Firefox, press F12 (Developer Tools)")
    print("2. Go to Storage tab")
    print("3. Click on Cookies -> https://www.minecraft-schematics.com")
    print("4. Copy the values for PHPSESSID, uid, ukey, utoken")
    print("5. Update the cookies_dict above and run again")

Starting fresh Firefox instance...
✓ Firefox started
Opening minecraft-schematics.com...

Adding your login cookies...
  Added cookie: PHPSESSID
  Added cookie: uid
  Added cookie: ukey
  Added cookie: utoken

Refreshing page to apply cookies...

✓✓✓ Successfully logged in with cookies!
Ready to start scraping!


In [25]:
# Test download with Selenium after you've logged in
test_id = 1857

print(f"Testing download for schematic {test_id}...")
driver.get(f"https://www.minecraft-schematics.com/schematic/{test_id}/")

# Wait for page to load
time.sleep(2)

# Check if we're logged in by looking for login/logout links
page_source = driver.page_source
if 'logout' in page_source.lower():
    print("✓ You are logged in!")
elif 'login' in page_source.lower():
    print("✗ Not logged in yet. Please login in the browser window.")
else:
    print("? Could not determine login status")

print(f"\nCurrent URL: {driver.current_url}")
print(f"Page title: {driver.title}")

# Try to download
download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/"
print(f"\nNavigating to download URL: {download_url}")
driver.get(download_url)

time.sleep(2)

final_url = driver.current_url
print(f"Final URL: {final_url}")

if 'login' in final_url:
    print("\n✗ Redirected to login - please log in manually first")
else:
    print("\n✓ Download initiated (check your Downloads folder)")
    print("If the file downloaded successfully, we can proceed with the full scraper!")

Testing download for schematic 1857...
✓ You are logged in!

Current URL: https://www.minecraft-schematics.com/schematic/1857/
Page title: Obi's Pokecenter, creation #1857

Navigating to download URL: https://www.minecraft-schematics.com/schematic/1857/download/
Final URL: https://www.minecraft-schematics.com/schematic/1857/download/

✓ Download initiated (check your Downloads folder)
If the file downloaded successfully, we can proceed with the full scraper!


In [47]:
# Full Selenium-based scraper
from pathlib import Path
import csv
from bs4 import BeautifulSoup
import shutil
import os

def selenium_scraper(driver, start_id, end_id, delay=1.5):
    """
    Scrape schematics using Selenium with an authenticated browser session.
    
    Args:
        driver: Selenium WebDriver instance (already logged in)
        start_id: Starting schematic ID
        end_id: Ending schematic ID
        delay: Delay between requests in seconds
    """
    
    # Setup paths
    PROJECT_ROOT = Path.cwd().parent
    DATA_DIR = PROJECT_ROOT / "data"
    OUTPUT_FOLDER = DATA_DIR / "schematics"
    METADATA_FILE = DATA_DIR / "schematics_metadata.csv"
    DOWNLOADS_FOLDER = Path.home() / "Downloads"
    
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
    print(f"Output folder: {OUTPUT_FOLDER}")
    print(f"Downloads folder: {DOWNLOADS_FOLDER}")
    
    # CSV headers
    csv_headers = ['id', 'title', 'category', 'theme', 'size', 'submitted_by', 
                   'posted_on', 'downloads', 'description', 'local_filename', 'status']
    
    successful = 0
    failed = 0
    not_found = 0
    
    with open(METADATA_FILE, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_headers)
        writer.writeheader()
        
        for schematic_id in range(start_id, end_id + 1):
            print(f"\n{'='*60}")
            print(f"Processing Schematic ID: {schematic_id}")
            print(f"{'='*60}")
            
            page_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/"
            
            try:
                # Navigate to schematic page
                driver.get(page_url)
                time.sleep(1)  # Wait for page load
                
                # Check if page exists (look at current URL after any redirects)
                current_url = driver.current_url
                
                # If redirected away, it might not exist
                if str(schematic_id) not in current_url:
                    print(f"✗ Schematic not found (redirected to {current_url})")
                    metadata = {
                        'id': schematic_id,
                        'title': 'N/A', 'category': 'N/A', 'theme': 'N/A',
                        'size': 'N/A', 'submitted_by': 'N/A', 'posted_on': 'N/A',
                        'downloads': '0', 'description': 'N/A',
                        'local_filename': 'N/A', 'status': 'not_found'
                    }
                    writer.writerow(metadata)
                    not_found += 1
                    time.sleep(delay)
                    continue
                
                # Parse page with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # Extract metadata
                def get_info_from_table(label):
                    label_tag = soup.find('strong', string=lambda x: x and label in x if x else False)
                    if label_tag and label_tag.parent:
                        value_cell = label_tag.parent.find_next_sibling('td')
                        if value_cell:
                            return value_cell.get_text(strip=True)
                    return 'N/A'
                
                metadata = {
                    'id': schematic_id,
                    'title': soup.find('h1').get_text(strip=True) if soup.find('h1') else 'N/A',
                    'category': get_info_from_table('Category'),
                    'theme': get_info_from_table('Theme'),
                    'size': get_info_from_table('Size'),
                    'submitted_by': get_info_from_table('Submitted by'),
                    'posted_on': get_info_from_table('Posted on'),
                    'downloads': '0',
                    'description': 'N/A',
                    'local_filename': 'N/A',
                    'status': 'pending'
                }
                
                # Extract downloads count
                downloads_text = get_info_from_table('Download')
                if 'downloaded' in downloads_text:
                    import re
                    match = re.search(r'(\d+)', downloads_text)
                    if match:
                        metadata['downloads'] = match.group(1)
                
                # Extract description - look for p tag in span10 divs with substantial content
                span10_divs = soup.find_all('div', class_='span10')
                for div in span10_divs:
                    p_tags = div.find_all('p')
                    for p in p_tags:
                        text = p.get_text(strip=True)
                        # Check if this looks like a description (substantial content)
                        if len(text) > 30 and not text.startswith('Category:') and not text.startswith('Theme:'):
                            metadata['description'] = text
                            break
                    if metadata['description'] != 'N/A':
                        break
                
                print(f"Title: {metadata['title']}")
                print(f"Category: {metadata['category']}")
                print(f"Size: {metadata['size']}")
                
                # Now download the file
                download_page_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/download/"
                print(f"\nNavigating to download page...")
                
                driver.get(download_page_url)
                time.sleep(1.5)  # Wait for page to load
                
                # Check if we were redirected to login
                if 'login' in driver.current_url:
                    print("✗ Redirected to login - session expired!")
                    metadata['status'] = 'auth_required'
                    writer.writerow(metadata)
                    csvfile.flush()
                    failed += 1
                    break  # Stop scraping, need to re-login
                
                # Find and click the actual download link
                try:
                    download_link = driver.find_element("xpath", "//a[contains(text(), 'Download Schematic File')]")
                    download_link.click()
                    time.sleep(2)  # Wait for download to complete
                    
                    # File will go to Downloads folder by default
                    filename = f"{schematic_id}.schematic"
                    source_path = DOWNLOADS_FOLDER / filename
                    dest_path = OUTPUT_FOLDER / filename
                    
                    # Move file from Downloads to project folder
                    if source_path.exists():
                        shutil.move(str(source_path), str(dest_path))
                        print(f"✓ Downloaded and moved: {filename}")
                        metadata['local_filename'] = filename
                        metadata['status'] = 'downloaded'
                        writer.writerow(metadata)
                        csvfile.flush()
                        successful += 1
                    else:
                        print(f"⚠ Download completed but file not found at: {source_path}")
                        metadata['status'] = 'file_not_found'
                        writer.writerow(metadata)
                        csvfile.flush()
                        failed += 1
                        
                except Exception as click_error:
                    print(f"✗ Could not click download link: {click_error}")
                    metadata['status'] = 'download_failed'
                    writer.writerow(metadata)
                    csvfile.flush()
                    failed += 1
                
            except Exception as e:
                print(f"✗ Error: {e}")
                metadata = {
                    'id': schematic_id,
                    'title': 'N/A', 'category': 'N/A', 'theme': 'N/A',
                    'size': 'N/A', 'submitted_by': 'N/A', 'posted_on': 'N/A',
                    'downloads': '0', 'description': 'N/A',
                    'local_filename': 'N/A', 'status': 'error'
                }
                writer.writerow(metadata)
                failed += 1
            
            # Be respectful with delays
            time.sleep(delay)
    
    print(f"\n{'='*60}")
    print("SCRAPING COMPLETE")
    print(f"{'='*60}")
    print(f"✓ Successful: {successful}")
    print(f"✗ Not Found: {not_found}")
    print(f"✗ Failed: {failed}")
    print(f"\nFiles saved to: {OUTPUT_FOLDER}")
    print(f"Metadata saved to: {METADATA_FILE}")

# Don't run yet - run this after confirming login works
print("Function defined. To start scraping, run:")
print("selenium_scraper(driver, START_ID, END_ID)")
print("\nExample: selenium_scraper(driver, 1824, 1834)  # Test with 10 items first")

Function defined. To start scraping, run:
selenium_scraper(driver, START_ID, END_ID)

Example: selenium_scraper(driver, 1824, 1834)  # Test with 10 items first


In [32]:
# Improved scraper that actually downloads files using requests with Selenium cookies
import requests
from pathlib import Path
import csv
from bs4 import BeautifulSoup
import time

def selenium_scraper_v2(driver, start_id, end_id, delay=1.5):
    """
    Scrape schematics using Selenium for auth + requests for downloading.
    """
    
    # Setup paths
    PROJECT_ROOT = Path.cwd().parent
    DATA_DIR = PROJECT_ROOT / "data"
    OUTPUT_FOLDER = DATA_DIR / "schematics"
    METADATA_FILE = DATA_DIR / "schematics_metadata.csv"
    
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
    print(f"Output folder: {OUTPUT_FOLDER}")
    
    # Create a requests session with Selenium cookies
    session = requests.Session()
    
    # Transfer cookies from Selenium to requests
    print("Transferring authentication cookies...")
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    print(f"Cookies transferred: {list(session.cookies.keys())}")
    
    # Setup headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }
    
    # CSV headers
    csv_headers = ['id', 'title', 'category', 'theme', 'size', 'submitted_by', 
                   'posted_on', 'downloads', 'description', 'local_filename', 'status']
    
    successful = 0
    failed = 0
    not_found = 0
    
    with open(METADATA_FILE, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_headers)
        writer.writeheader()
        
        for schematic_id in range(start_id, end_id + 1):
            print(f"\n{'='*60}")
            print(f"Processing Schematic ID: {schematic_id} ({schematic_id - start_id + 1}/{end_id - start_id + 1})")
            print(f"{'='*60}")
            
            page_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/"
            
            try:
                # Get page with requests
                headers['Referer'] = page_url
                response = session.get(page_url, headers=headers, timeout=10)
                
                # Check if page exists
                if response.status_code == 404 or 'not found' in response.text.lower():
                    print(f"✗ Schematic not found")
                    metadata = {
                        'id': schematic_id,
                        'title': 'N/A', 'category': 'N/A', 'theme': 'N/A',
                        'size': 'N/A', 'submitted_by': 'N/A', 'posted_on': 'N/A',
                        'downloads': '0', 'description': 'N/A',
                        'local_filename': 'N/A', 'status': 'not_found'
                    }
                    writer.writerow(metadata)
                    csvfile.flush()  # Save immediately
                    not_found += 1
                    time.sleep(delay)
                    continue
                
                # Parse page
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Extract metadata
                def get_info_from_table(label):
                    label_tag = soup.find('strong', string=lambda x: x and label in x if x else False)
                    if label_tag and label_tag.parent:
                        value_cell = label_tag.parent.find_next_sibling('td')
                        if value_cell:
                            return value_cell.get_text(strip=True)
                    return 'N/A'
                
                metadata = {
                    'id': schematic_id,
                    'title': soup.find('h1').get_text(strip=True) if soup.find('h1') else 'N/A',
                    'category': get_info_from_table('Category'),
                    'theme': get_info_from_table('Theme'),
                    'size': get_info_from_table('Size'),
                    'submitted_by': get_info_from_table('Submitted by'),
                    'posted_on': get_info_from_table('Posted on'),
                    'downloads': '0',
                    'description': 'N/A',
                    'local_filename': 'N/A',
                    'status': 'pending'
                }
                
                # Extract downloads count
                downloads_text = get_info_from_table('Download')
                if 'downloaded' in downloads_text:
                    import re
                    match = re.search(r'(\d+)', downloads_text)
                    if match:
                        metadata['downloads'] = match.group(1)
                
                # Extract description
                desc_div = soup.find('div', class_=lambda x: x and 'description' in x.lower() if x else False)
                if desc_div:
                    metadata['description'] = desc_div.get_text(strip=True)
                
                print(f"  Title: {metadata['title']}")
                print(f"  Category: {metadata['category']}")
                print(f"  Size: {metadata['size']}")
                
                # Download the file using the simple download URL
                download_url = f"https://www.minecraft-schematics.com/schematic/{schematic_id}/download/"
                print(f"  Downloading from: {download_url}")
                
                download_response = session.get(download_url, headers=headers, timeout=15, allow_redirects=True)
                
                # Check content type
                content_type = download_response.headers.get('content-type', '')
                
                if 'text/html' in content_type:
                    print(f"  ✗ Got HTML instead of file (status: {download_response.status_code})")
                    if 'login' in download_response.url:
                        print("  ⚠ Session expired - stopping")
                        metadata['status'] = 'auth_expired'
                        writer.writerow(metadata)
                        csvfile.flush()
                        break
                    metadata['status'] = 'no_file'
                    writer.writerow(metadata)
                    csvfile.flush()
                    failed += 1
                else:
                    # Save the file
                    filename = f"{schematic_id}.schematic"
                    filepath = OUTPUT_FOLDER / filename
                    
                    with open(filepath, 'wb') as f:
                        f.write(download_response.content)
                    
                    file_size = len(download_response.content)
                    print(f"  ✓ Downloaded: {filename} ({file_size} bytes)")
                    
                    metadata['local_filename'] = filename
                    metadata['status'] = 'downloaded'
                    writer.writerow(metadata)
                    csvfile.flush()  # Save immediately
                    successful += 1
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
                metadata = {
                    'id': schematic_id,
                    'title': 'N/A', 'category': 'N/A', 'theme': 'N/A',
                    'size': 'N/A', 'submitted_by': 'N/A', 'posted_on': 'N/A',
                    'downloads': '0', 'description': 'N/A',
                    'local_filename': 'N/A', 'status': 'error'
                }
                writer.writerow(metadata)
                csvfile.flush()
                failed += 1
            
            time.sleep(delay)
    
    print(f"\n{'='*60}")
    print("SCRAPING COMPLETE")
    print(f"{'='*60}")
    print(f"✓ Successful: {successful}")
    print(f"✗ Not Found: {not_found}")
    print(f"✗ Failed: {failed}")
    print(f"\nFiles saved to: {OUTPUT_FOLDER}")
    print(f"Metadata saved to: {METADATA_FILE}")

print("Improved scraper function defined!")
print("Test it with: selenium_scraper_v2(driver, 1824, 1834)")

Improved scraper function defined!
Test it with: selenium_scraper_v2(driver, 1824, 1834)


In [33]:
# Test the improved scraper
print("Testing improved scraper with actual file downloads...")
selenium_scraper_v2(driver, 1857, 1860)  # Test with just 3-4 items

Testing improved scraper with actual file downloads...
Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Transferring authentication cookies...
Cookies transferred: ['PHPSESSID', 'uid', 'ukey', 'utoken', 'FCCDCF', '__gads', '__gpi', '__eoi', 'FCNEC', '_awl', 'cf_clearance']

Processing Schematic ID: 1857 (1/4)
  Title: Obi's Pokecenter
  Category: Houses And Shops
  Size: Medium
  ✗ Got HTML instead of file (status: 200)
  ⚠ Session expired - stopping

SCRAPING COMPLETE
✓ Successful: 0
✗ Not Found: 0
✗ Failed: 0

Files saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Metadata saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv


In [34]:
# Test run with a small range (10 schematics)
print("Starting test run with 10 schematics...")
selenium_scraper(driver, 1824, 1834)

Starting test run with 10 schematics...
Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics

Processing Schematic ID: 1824
Title: greek temple
Category: Temples
Size: Medium

Initiating download...
✓ Download initiated for 1824.schematic

Processing Schematic ID: 1825
✗ Schematic not found (redirected to https://www.minecraft-schematics.com/)

Processing Schematic ID: 1826
✗ Schematic not found (redirected to https://www.minecraft-schematics.com/)

Processing Schematic ID: 1827
Title: Docked Dreadnought
Category: Miscellaneous
Size: Small

Initiating download...
✓ Download initiated for 1827.schematic

Processing Schematic ID: 1828
Title: green villager spawner
Category: Houses And Shops
Size: Small

Initiating download...
✓ Download initiated for 1828.schematic

Processing Schematic ID: 1829
Title: Mansion by Alurra17
Category: Houses And Shops
Size: Large

Initiating download...
✓ Download initiated for 1829.schematic

Processing Schematic ID: 1830
Title: Ballibury

In [35]:
# Debug: Check what happens when we navigate to download URL
test_id = 1857
print(f"Navigating to download URL for schematic {test_id}...")
print(f"Watch the Firefox window!\n")

download_url = f"https://www.minecraft-schematics.com/schematic/{test_id}/download/"
print(f"URL: {download_url}")

driver.get(download_url)
time.sleep(3)

print(f"\nFinal URL: {driver.current_url}")
print(f"Page title: {driver.title}")

# Check if we're on a login page
if 'login' in driver.current_url.lower():
    print("\n⚠ We were redirected to LOGIN page!")
    print("This means the Selenium session is NOT authenticated.")
elif driver.current_url == download_url or 'download' in driver.current_url:
    print("\n✓ Still on download URL")
    # Let's see what the page shows
    page_text = driver.page_source[:500]
    print(f"\nFirst 500 chars of page source:")
    print(page_text)

Navigating to download URL for schematic 1857...
Watch the Firefox window!

URL: https://www.minecraft-schematics.com/schematic/1857/download/

Final URL: https://www.minecraft-schematics.com/schematic/1857/download/
Page title: Download Obi's Pokecenter, creation #1857

✓ Still on download URL

First 500 chars of page source:
<html lang="en"><head>
	<meta charset="utf-8">
	<title>Download Obi's Pokecenter, creation #1857</title>
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<meta name="description" content="">	<meta name="keywords" content="">	<meta name="author" content="Listforge">
	<link href="/assets/css/bootstrap-combined.min.css" rel="stylesheet">
	<link href="https://cdn.jsdelivr.net/npm/font-awesome@4.7.0/css/font-awesome.min.css" rel="stylesheet">
	<link href="/assets/css/styles.css"


In [36]:
# Look for download buttons/links on the download page
soup = BeautifulSoup(driver.page_source, 'html.parser')

print("Looking for download links/buttons...\n")

# Look for links containing "download"
download_links = soup.find_all('a', href=True)
for link in download_links:
    href = link.get('href', '')
    text = link.get_text(strip=True)
    if 'download' in href.lower() or 'download' in text.lower() or 'schematic' in href.lower():
        print(f"Link: {text}")
        print(f"Href: {href}")
        print(f"---")

# Look for buttons
buttons = soup.find_all('button')
for button in buttons:
    print(f"Button: {button.get_text(strip=True)}")
    print(f"Attributes: {button.attrs}")
    print(f"---")

# Look for forms
forms = soup.find_all('form')
print(f"\nFound {len(forms)} forms")
for i, form in enumerate(forms):
    print(f"\nForm {i+1}:")
    print(f"Action: {form.get('action')}")
    print(f"Method: {form.get('method')}")
    print(f"Contents: {form.get_text(strip=True)[:100]}")

Looking for download links/buttons...

Link: Most Downloaded
Href: /most-downloaded/
---
Link: Post Your Creation
Href: /schematic/add/
---
Link: Post Your Creation
Href: /schematic/add/
---
Link: Your Creations
Href: /schematics/manage/
---
Link: Your Downloads
Href: /downloads/
---
Link: Obi's Pokecenter
Href: /schematic/1857/
---
Link: Schematic Details
Href: /schematic/1857/
---
Link: Download Schematic File
Href: /schematic/1857/download/action/?type=schematic
---
Link: 
Href: /schematic/5987/
---
Link: 
Href: /schematic/1866/
---
Link: 
Href: /schematic/5988/
---
Link: 
Href: /schematic/1868/
---
Link: Most Downloaded
Href: /most-downloaded/
---
Link: 
Href: https://twitter.com/MineSchematics
---

Found 0 forms


In [37]:
# Test clicking the actual download link
print("Clicking 'Download Schematic File' link...")

# Find and click the link
download_link = driver.find_element("xpath", "//a[contains(text(), 'Download Schematic File')]")
print(f"Found link, clicking...")

download_link.click()

print("Click performed! Waiting 3 seconds...")
time.sleep(3)

print(f"\nCurrent URL after click: {driver.current_url}")

# Check if file downloaded
import os
downloads_path = os.path.expanduser("~/Downloads")
schematic_files = [f for f in os.listdir(downloads_path) if f.endswith('.schematic')]
if schematic_files:
    print(f"\n✓ SUCCESS! Found .schematic files in Downloads:")
    for f in schematic_files[-5:]:  # Show last 5
        full_path = os.path.join(downloads_path, f)
        size = os.path.getsize(full_path)
        print(f"  - {f} ({size} bytes)")
else:
    print("\n✗ No .schematic files found in Downloads yet")

Clicking 'Download Schematic File' link...
Found link, clicking...
Click performed! Waiting 3 seconds...

Current URL after click: https://www.minecraft-schematics.com/schematic/1857/download/

✓ SUCCESS! Found .schematic files in Downloads:
  - 1857.schematic (1271 bytes)


In [39]:
# Test the updated selenium_scraper with clicking
print("Testing updated selenium_scraper with actual clicking...")
print("This will download files to ~/Downloads/\n")

# Test with just 3 schematics first
selenium_scraper(driver, 1858, 1860)

Testing updated selenium_scraper with actual clicking...
This will download files to ~/Downloads/

Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics

Processing Schematic ID: 1858
Title: Obi's Pokemart
Category: Houses And Shops
Size: Medium

Navigating to download page...
✓ Downloaded: 1858.schematic

Processing Schematic ID: 1859
Title: Grand Colosseum II
Category: Arenas
Size: Huge

Navigating to download page...
✓ Downloaded: 1859.schematic

Processing Schematic ID: 1860
Title: Small Brick House
Category: Houses And Shops
Size: Small

Navigating to download page...
✓ Downloaded: 1860.schematic

SCRAPING COMPLETE
✓ Successful: 3
✗ Not Found: 0
✗ Failed: 0

Metadata saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv

⚠ NOTE: Files were downloaded to ~/Downloads/
To move them to the project folder, run:
  mv ~/Downloads/*.schematic /Users/moustholmes/minecraft_voxel_flow/data/schematics/


In [40]:
# Test the updated scraper with automatic file moving
print("Testing scraper with automatic file moving...")
print("Files will be automatically moved from ~/Downloads/ to project folder\n")

# Test with 2 new schematics
selenium_scraper(driver, 1861, 1862)

Testing scraper with automatic file moving...
Files will be automatically moved from ~/Downloads/ to project folder

Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics

Processing Schematic ID: 1861
Title: Sinking Ship by LecoRubb
Category: Boats
Size: Large

Navigating to download page...
✓ Downloaded: 1861.schematic

Processing Schematic ID: 1862
Title: Medieval-Crane #2
Category: Towers
Size: Medium

Navigating to download page...
✓ Downloaded: 1862.schematic

SCRAPING COMPLETE
✓ Successful: 2
✗ Not Found: 0
✗ Failed: 0

Metadata saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv

⚠ NOTE: Files were downloaded to ~/Downloads/
To move them to the project folder, run:
  mv ~/Downloads/*.schematic /Users/moustholmes/minecraft_voxel_flow/data/schematics/


In [42]:
# Test with automatic moving (reloaded function)
print("Testing RELOADED scraper with automatic file moving...")
selenium_scraper(driver, 1863, 1864)

Testing RELOADED scraper with automatic file moving...
Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Downloads folder: /Users/moustholmes/Downloads

Processing Schematic ID: 1863
Title: Farmhouse
Category: Houses And Shops
Size: Large

Navigating to download page...
✓ Downloaded and moved: 1863.schematic

Processing Schematic ID: 1864
Title: Barn with Water Tower
Category: Houses And Shops
Size: Medium

Navigating to download page...
✓ Downloaded and moved: 1864.schematic

SCRAPING COMPLETE
✓ Successful: 2
✗ Not Found: 0
✗ Failed: 0

Files saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Metadata saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv


In [43]:
# Debug: Check the description extraction for schematic 1864
test_url = "https://www.minecraft-schematics.com/schematic/1864/"
driver.get(test_url)
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')

print("Looking for description...")
print("\n1. Trying class='description':")
desc_div = soup.find('div', class_='description')
print(f"Found: {desc_div}")

print("\n2. Trying class contains 'description':")
desc_div2 = soup.find('div', class_=lambda x: x and 'description' in x.lower() if x else False)
print(f"Found: {desc_div2}")

print("\n3. Looking for all divs with 'description' in text:")
all_divs = soup.find_all('div')
for div in all_divs:
    if div.get('class'):
        class_str = ' '.join(div.get('class'))
        if 'descr' in class_str.lower():
            print(f"Found div with class: {div.get('class')}")
            print(f"Content preview: {div.get_text(strip=True)[:200]}")
            print("---")

print("\n4. Looking for the actual description text:")
if 'READ !' in driver.page_source:
    print("✓ Found 'READ !' in page source")
    # Find the element containing it
    import re
    match = re.search(r'(READ.*?iEdgy.*?-{20,})', driver.page_source, re.DOTALL)
    if match:
        print(f"\nFound text: {match.group(0)[:200]}...")
else:
    print("✗ 'READ !' not found in page source")

Looking for description...

1. Trying class='description':
Found: None

2. Trying class contains 'description':
Found: None

3. Looking for all divs with 'description' in text:

4. Looking for the actual description text:
✓ Found 'READ !' in page source

Found text: READ !<br>
This creation was built on the server of Sharky009 !<br>
-&gt; Check it out IP: sharky009.no-ip.org and vist the orginal world !<br>
A lot of fun with our schematics :D<br>
<br>
~iEdgy~<br>...


In [44]:
# Find the HTML structure around the description
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Look for text containing "READ !"
import re
for element in soup.find_all(string=re.compile(r'READ\s*!')):
    print(f"Found in: {element.parent.name}")
    print(f"Parent classes: {element.parent.get('class')}")
    print(f"Parent parent: {element.parent.parent.name if element.parent.parent else 'None'}")
    print(f"Parent parent classes: {element.parent.parent.get('class') if element.parent.parent else 'None'}")
    
    # Get the full content
    content_element = element.parent
    while content_element and content_element.name != 'div':
        content_element = content_element.parent
    
    if content_element:
        print(f"\nClosest div: {content_element.get('class')}")
        print(f"Content: {content_element.get_text(strip=True)[:300]}")
    break

# Alternative: Look for "Description" label
print("\n\n=== Looking for Description label ===")
desc_label = soup.find('strong', string=lambda x: x and 'Description' in x if x else False)
if desc_label:
    print(f"Found Description label!")
    print(f"Label parent: {desc_label.parent.name}")
    
    # Try to find the content after the label
    content_area = desc_label.parent.find_next_sibling()
    if content_area:
        print(f"Content sibling: {content_area.name}")
        print(f"Content classes: {content_area.get('class')}")
        print(f"Content text: {content_area.get_text(strip=True)[:300]}")

Found in: p
Parent classes: None
Parent parent: div
Parent parent classes: ['span10']

Closest div: ['span10']
Content: Description------------------------------------------------------------------------------------------------------------------------------------READ !This creation was built on the server of Sharky009 !-> Check it out IP: sharky009.no-ip.org and vist the orginal world !A lot of fun with our schematic


=== Looking for Description label ===


In [45]:
# Better way: Look for the word "Description" in the page
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all text containing "Description"
for element in soup.find_all(string=re.compile(r'Description', re.IGNORECASE)):
    parent = element.parent
    print(f"Found 'Description' in: {parent.name}")
    print(f"Classes: {parent.get('class')}")
    print(f"Full text: {parent.get_text(strip=True)[:200]}")
    print("---")
    
    # If this is a heading/label, get the next sibling or content
    next_elem = parent.find_next_sibling()
    if next_elem:
        print(f"Next sibling: {next_elem.name}, classes: {next_elem.get('class')}")
        print(f"Text: {next_elem.get_text(strip=True)[:300]}")
    print("\n")

# Try looking for p tags in span10 divs
print("\n=== All p tags in span10 divs ===")
span10_divs = soup.find_all('div', class_='span10')
for div in span10_divs:
    p_tags = div.find_all('p')
    for p in p_tags:
        text = p.get_text(strip=True)
        if len(text) > 50:  # Only show substantial content
            print(f"P tag content: {text[:300]}")
            print("---")

Found 'Description' in: legend
Classes: None
Full text: Description
---



=== All p tags in span10 divs ===
P tag content: ------------------------------------------------------------------------------------------------------------------------------------READ !This creation was built on the server of Sharky009 !-> Check it out IP: sharky009.no-ip.org and vist the orginal world !A lot of fun with our schematics :D~iEdgy~
---


In [46]:
# Final solution: Extract description properly
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Method 1: Find legend with "Description" and get the next p tag
legend = soup.find('legend', string=re.compile(r'Description', re.IGNORECASE))
if legend:
    print("Method 1: Found legend")
    # Find the parent fieldset or containing div
    container = legend.parent
    print(f"Container: {container.name}, classes: {container.get('class')}")
    
    # Find p tag in the container
    p_tag = container.find('p')
    if p_tag:
        desc_text = p_tag.get_text(strip=True)
        print(f"\n✓ Description found:\n{desc_text}")
    else:
        print("No p tag found in container")
        
# Method 2: Find span10 div with a p tag that has substantial content
print("\n\nMethod 2: Find span10 with description content")
span10_divs = soup.find_all('div', class_='span10')
for div in span10_divs:
    p_tags = div.find_all('p')
    for p in p_tags:
        text = p.get_text(strip=True)
        # Check if this looks like a description (not just a short label)
        if len(text) > 30 and not text.startswith('Category:') and not text.startswith('Theme:'):
            print(f"✓ Found description in span10 p tag:\n{text[:300]}")
            break

Method 1: Found legend
Container: p, classes: None
No p tag found in container


Method 2: Find span10 with description content
✓ Found description in span10 p tag:
------------------------------------------------------------------------------------------------------------------------------------READ !This creation was built on the server of Sharky009 !-> Check it out IP: sharky009.no-ip.org and vist the orginal world !A lot of fun with our schematics :D~iEdgy~


In [48]:
# Test the fixed description extraction
print("Testing fixed description extraction on schematic 1864...")
selenium_scraper(driver, 1864, 1864)

Testing fixed description extraction on schematic 1864...
Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Downloads folder: /Users/moustholmes/Downloads

Processing Schematic ID: 1864
Title: Barn with Water Tower
Category: Houses And Shops
Size: Medium

Navigating to download page...
✓ Downloaded and moved: 1864.schematic

SCRAPING COMPLETE
✓ Successful: 1
✗ Not Found: 0
✗ Failed: 0

Files saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Metadata saved to: /Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv


In [49]:
# Test CSV parsing to make sure descriptions with commas, quotes, etc. are handled correctly
import csv

print("Testing CSV parsing...")
print("\nReading the metadata CSV file:")

with open('/Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['id'] == '1864':
            print(f"\nSchematic ID: {row['id']}")
            print(f"Title: {row['title']}")
            print(f"Category: {row['category']}")
            print(f"Description length: {len(row['description'])} characters")
            print(f"Description preview: {row['description'][:200]}...")
            print(f"\n✓ CSV parsing works correctly!")
            
            # Check if there are any issues
            if ',' in row['description']:
                print(f"✓ Comma in description handled correctly")
            if '"' in row['description']:
                print(f"✓ Quotes in description handled correctly")
            break

# Also test with a description that has problematic characters
print("\n\nTesting with problematic characters:")
test_desc = 'This has, commas, "quotes", and\nnewlines!'
print(f"Test description: {test_desc}")

# Write and read back
test_file = '/Users/moustholmes/minecraft_voxel_flow/data/test_csv.csv'
with open(test_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['id', 'description'])
    writer.writeheader()
    writer.writerow({'id': '999', 'description': test_desc})

with open(test_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['description'] == test_desc:
            print(f"✓ Problematic characters preserved correctly!")
        else:
            print(f"✗ Issue detected!")
            print(f"Original: {test_desc}")
            print(f"Read back: {row['description']}")

# Clean up test file
import os
os.remove(test_file)

Testing CSV parsing...

Reading the metadata CSV file:

Schematic ID: 1864
Title: Barn with Water Tower
Category: Houses And Shops
Description length: 432 characters
Description preview: ------------------------------------------------------------------------------------------------------------------------------------READ !This creation was built on the server of Sharky009 !-> Check i...

✓ CSV parsing works correctly!


Testing with problematic characters:
Test description: This has, commas, "quotes", and
newlines!
✓ Problematic characters preserved correctly!


In [50]:
# Check the actual CSV format with Python
import csv

print("Checking CSV structure:")
with open('/Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv', 'r', encoding='utf-8') as f:
    # Read raw lines
    lines = f.readlines()
    print(f"\nTotal lines in file: {len(lines)}")
    print(f"Header: {lines[0].strip()}")
    
    # Check if row 2 (1864) is properly formatted
    if len(lines) > 1:
        print(f"\nRow 2 preview (first 200 chars): {lines[1][:200]}")
        
# Now parse it properly with csv module
print("\n" + "="*60)
print("Parsing with CSV module (proper way):")
print("="*60)

with open('/Users/moustholmes/minecraft_voxel_flow/data/schematics_metadata.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    rows = list(reader)
    print(f"\nTotal rows parsed: {len(rows)}")
    
    for row in rows:
        print(f"\nID: {row['id']}")
        print(f"Title: {row['title']}")
        print(f"Category: {row['category']}")
        desc_preview = row['description'][:100] if len(row['description']) > 100 else row['description']
        print(f"Description (first 100 chars): {desc_preview}")
        print(f"Status: {row['status']}")
        print("-" * 60)

Checking CSV structure:

Total lines in file: 2
Header: id,title,category,theme,size,submitted_by,posted_on,downloads,description,local_filename,status

Row 2 preview (first 200 chars): 1864,Barn with Water Tower,Houses And Shops,Medieval,Medium,Chunkie,"November 17th, 2013 09:28 AM EST",1236,--------------------------------------------------------------------------------------------

Parsing with CSV module (proper way):

Total rows parsed: 1

ID: 1864
Title: Barn with Water Tower
Category: Houses And Shops
Description (first 100 chars): ----------------------------------------------------------------------------------------------------
Status: downloaded
------------------------------------------------------------


In [ ]:
# FULL SCRAPING RUN
# This will scrape from ID 1824 to 19000 (approximately 17,176 schematics)
# Estimated time: ~12-14 hours (at ~2.5 seconds per schematic)

import time
start_time = time.time()

print("="*70)
print("STARTING FULL SCRAPING RUN")
print("="*70)
print(f"Range: 1824 to 19000")
print(f"Estimated total: ~17,176 schematics")
print(f"Estimated time: 12-14 hours")
print(f"Started at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)
print("\nYou can safely close this browser window - the scraping will continue.")
print("The Firefox window needs to stay open for authentication.\n")

# Run the scraper
selenium_scraper(driver, 1824, 19000)

# Calculate elapsed time
elapsed = time.time() - start_time
hours = int(elapsed // 3600)
minutes = int((elapsed % 3600) // 60)
seconds = int(elapsed % 60)

print(f"\n{'='*70}")
print(f"SCRAPING COMPLETED!")
print(f"Total time: {hours}h {minutes}m {seconds}s")
print(f"Finished at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*70}")

STARTING FULL SCRAPING RUN
Range: 1824 to 19000
Estimated total: ~17,176 schematics
Estimated time: 12-14 hours
Started at: 2025-10-07 13:37:41

You can safely close this browser window - the scraping will continue.
The Firefox window needs to stay open for authentication.

Output folder: /Users/moustholmes/minecraft_voxel_flow/data/schematics
Downloads folder: /Users/moustholmes/Downloads

Processing Schematic ID: 1824
Title: greek temple
Category: Temples
Size: Medium

Navigating to download page...
✓ Downloaded and moved: 1824.schematic

Processing Schematic ID: 1825
✗ Schematic not found (redirected to https://www.minecraft-schematics.com/)

Processing Schematic ID: 1826
✗ Schematic not found (redirected to https://www.minecraft-schematics.com/)

Processing Schematic ID: 1827
Title: Docked Dreadnought
Category: Miscellaneous
Size: Small

Navigating to download page...
✗ Could not click download link: Message: Unable to locate element: //a[contains(text(), 'Download Schematic File')

In [ ]:
# Full run: scrape all schematics from 1824 to 19000
print("Starting FULL scraping run...")
print("This will take several hours. Progress will be shown for each schematic.")
print("\nYou can stop at any time with Ctrl+C, and the CSV will have all completed entries.")
print("="*60)

selenium_scraper(driver, 1824, 19000)

In [6]:
main()

Created directory: /Users/moustholmes/minecraft_voxel_flow/data/schematics

Attempting to login...
✗ Login failed. Please check your credentials.

Continuing without login. Downloads may fail if authentication is required.
✗ Login failed. Please check your credentials.

Continuing without login. Downloads may fail if authentication is required.
Scraping cancelled.
Scraping cancelled.
